In [ ]:
import pandas as pd
import re
from psaw import PushshiftAPI
api = PushshiftAPI()

In [ ]:
politics_keywords = api.search_submissions(title='(India&President)|(Joe Biden&Politics)|(USA&Politics)|(President&elections)|(Kamala Harris&USA)', 
    filter=['id','url','author', 'title', 'score',
     'subreddit','selftext','num_comments', 'nest_level'], # list of fields to return
    limit = 800, # limit on the number of records returned
    num_comments=">20"
)

In [ ]:
politics = api.search_submissions(subreddit=['politics'], 
    filter=['id','url','author', 'title', 'score',
    'subreddit','selftext','num_comments'], # list of fields to return
    limit = 400, # limit on the number of records returned
    num_comments=">20")

In [ ]:
politics_df = pd.DataFrame(list(politics))
politics_kw_df = pd.DataFrame(list(politics_keywords))

In [ ]:
environment_kw = api.search_submissions(title='(nature)|(Pollution)|(Biodegradable)|(Recycle)|(Climate Change)|(fossil fuel)|(forests)', selftext='(nature)|(Pollution)|(Biodegradable)|(Recycle)|(Climate Change)|(fossil fuel)|(forests)',
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
environment = api.search_submissions(subreddit=['environment'],
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
environment_df = pd.DataFrame(list(environment))
environment_kw_df = pd.DataFrame(list(environment_kw))

In [ ]:
technology_kw = api.search_submissions(title='(Artificial Intelligence)|(Cybersecurity)|(Internet)|(Gadgets)|(Facebook)', selftext='(Artificial Intelligence)|(Cybersecurity)|(Internet)|(Gadgets)|(Facebook)',
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
technology = api.search_submissions(subreddit=['technology'],
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">25"
                                  )

In [ ]:
technology_kw_df = pd.DataFrame(list(technology_kw))
technology_df = pd.DataFrame(list(technology))

In [ ]:
healthcare_kw = api.search_submissions(title='(covid)|(healthcare)|(medical)|(hospital)|(dermatologist)|(surgery)', selftext='(covid)|(healthcare)|(medical)|(hospital)|(dermatologist)|(surgery)',
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
healthcare = api.search_submissions(subreddit=['healthcare'],
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
healthcare_kw_df = pd.DataFrame(list(healthcare_kw))
healthcare_df = pd.DataFrame(list(healthcare))

In [ ]:
education_kw = api.search_submissions(title='(education)|(scholarship)|(grades)|(curriculum)|(professors)',selftext='(education)|(scholarship)|(grades)|(curriculum)|(professors)',
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
education = api.search_submissions(subreddit=['education'],
                                  filter=['id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                  limit = 400, # limit on the number of records returned
                                  num_comments=">20"
                                  )

In [ ]:
education_kw_df = pd.DataFrame(list(education_kw))
education_df = pd.DataFrame(list(education))

In [ ]:
education_df = pd.concat([education_df, education_kw_df])
healthcare_df = pd.concat([healthcare_df, healthcare_kw_df])
technology_df = pd.concat([technology_df, technology_kw_df])
environment_df = pd.concat([environment_df, environment_kw_df])
politics_df = pd.concat([politics_df, politics_kw_df])
education_df['topic'] = 'Education'
healthcare_df['topic'] = 'Healthcare'
technology_df['topic'] = 'Technology'
environment_df['topic'] = 'Environment'
politics_df['topic'] = 'Politics'
topic_df = pd.concat([politics_df.iloc[:500], environment_df.iloc[:500], technology_df.iloc[:500], healthcare_df.iloc[:500], education_df.iloc[:500]])

In [ ]:
import datetime
topic_df['created_at'] = topic_df['created_utc'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%dT%H:%M:%SZ'))
all_submission = topic_df

In [ ]:
import datetime
topic_df['created_at'] = topic_df['created_utc'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%dT%H:%M:%SZ'))
all_submission = topic_df
topic_df[~topic_df['selftext'].apply(lambda x: '[deleted]' in x or '[removed]' in x)]
all_submission = all_submission[~all_submission['title'].apply(lambda x: '[delted]' in x or '[removed]' in x)]
all_submission['selftext'] = all_submission['selftext'].apply(lambda x: '' if '[delted]' in x or '[removed]' in x else x)


In [ ]:
count = 0
def get_comments(r_id, sub_reddit):
  global count
  if count % 20 == 0:
    print(count)
  comments_lst = list(api.search_comments(link_id=r_id, subreddit=[sub_reddit],
                                          filter=['id','parent_id','parent_body', 'permalink','author', 'title', 
                                                  'subreddit','body','num_comments','score', 'nest_level'], limit=100))
  count += 1
  return list(comments_lst)

In [ ]:
all_submission.shape

In [ ]:
all_comments_1 = all_submission[['id', 'subreddit']].iloc[:500].apply(lambda x: get_comments(x[0], x[1]), axis=1)
# print("---------------------------------------------------------------------------------------------------------")
all_comments_2 = all_submission[['id', 'subreddit']].iloc[500:1000].apply(lambda x: get_comments(x[0], x[1]), axis=1)
# print("---------------------------------------------------------------------------------------------------------")
all_comments_3 = all_submission[['id', 'subreddit']].iloc[1000:1500].apply(lambda x: get_comments(x[0], x[1]), axis=1)
# print("---------------------------------------------------------------------------------------------------------")
all_comments_4 = all_submission[['id', 'subreddit']].iloc[1500:2000].apply(lambda x: get_comments(x[0], x[1]), axis=1)
# print("---------------------------------------------------------------------------------------------------------")
all_comments_5 = all_submission[['id', 'subreddit']].iloc[2000:2500].apply(lambda x: get_comments(x[0], x[1]), axis=1)

In [ ]:
flag = 0
concat_comments = []
for r in pd.concat([all_comments_1, all_comments_2, all_comments_3, all_comments_4, all_comments_5]):
    try:
        concat_comments.append(pd.DataFrame(r))
    except:
        print(flag)
    flag+=1

In [ ]:
pd.concat([all_comments_1, all_comments_2, all_comments_3, all_comments_4, all_comments_5])
merged_comments = pd.concat(concat_comments)
merged_comments

In [ ]:
length = 0
ignore = [1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1792, 1793, 1794, 1796, 1798, 1799, 1800, 1805, 1806, 1813, 1814, 1818, 1820, 1821, 1830, 1832]
df_index = 0
id_topic_map = {}
for comments in [all_comments_1, all_comments_2, all_comments_3, all_comments_4, all_comments_5]:
    for index, comment in comments.iteritems():
        if df_index in ignore:
            df_index += 1
            continue
#         import ipdb;ipdb.set_trace()
        id_topic_map.update(dict(zip(range(length, length+len(comment)), [all_submission.iloc[df_index]['topic']]*len(comment))))
        length += len(comment)
        df_index +=1


In [ ]:
merged_comments['Topics'] = merged_comments.index.map(lambda x: id_topic_map[x])

In [ ]:
from collections import Counter
Counter(merged_comments.reset_index().index.map(lambda x: id_topic_map[x]))

In [ ]:
merged_comments

In [ ]:
merged_comments['Topics'].value_counts()

In [ ]:
all_submission.topic.value_counts()

In [ ]:
merged_comments

In [ ]:
valid_comments = merged_comments[~merged_comments['parent_id'].apply(lambda x: isinstance(x, int))]
parent_ids_t1 = valid_comments[valid_comments['parent_id'].apply(lambda x: x.split("_")[0] == 't1')]['parent_id']
parent_ids_t3 = valid_comments[valid_comments['parent_id'].apply(lambda x: x.split("_")[0] == 't3')]['parent_id']
set(parent_ids_t1.apply(lambda x: x.split('_')[-1]).to_list()).intersection(set(valid_comments['id'].to_list()))

In [ ]:
comments_not_present = set(parent_ids_t1.apply(lambda x: x.split('_')[-1]).to_list()).difference(set(valid_comments['id'].to_list()))
set(parent_ids_t3.apply(lambda x: x.split('_')[-1]).to_list()).difference(set(all_submission['id'].to_list()))

In [ ]:
import requests
count =0 
import time
def get_data(url):
    global count
    if count % 2000 == 0:
        time.sleep(5)
    a = requests.get(url)
    count += 500
    return a.json()

In [ ]:
comments_not_present = list(comments_not_present)
body_list = []
for i in range(0, len(comments_not_present), 500):
  body_list.append(get_data(f"https://api.pushshift.io/reddit/comment/search/?ids={','.join(comments_not_present[i:i+500])}"))

In [ ]:
mapping = {'id': [], 'body': []}
for res in body_list:
  for d in res['data']:
    mapping['id'].append(d['id'])
    mapping['body'].append(d.get('body'))

In [ ]:
all_comments_merged = pd.concat([valid_comments, pd.DataFrame.from_dict(mapping)])

In [ ]:
id_title_map = all_submission[['id', 'title']].set_index('id').to_dict()['title']

In [ ]:
id_selftext_map = all_submission[['id', 'selftext']].set_index('id').to_dict()['selftext']

In [ ]:
id_body_map = all_comments_merged[['id', 'body']].set_index('id').to_dict()['body']

In [ ]:
all_comments_merged['parent_title'] = all_comments_merged['parent_id'].apply(lambda x: id_title_map[x.split('_')[1]] if str(x).split('_')[0] == 't3' else '')
all_comments_merged['parent_selftext'] = all_comments_merged['parent_id'].apply(lambda x: id_selftext_map[x.split('_')[1]] if str(x).split('_')[0] == 't3' else '')
all_comments_merged['parent_body'] = all_comments_merged['parent_id'].apply(lambda x: id_body_map.get(x.split('_')[1], 'Missing') if str(x).split('_')[0] == 't1' else '')
all_comments_merged[['body', 'parent_title', 'parent_selftext', 'parent_body']]

In [ ]:
all_comments_merged['parent_main_body'] = all_comments_merged[['parent_body', 'parent_title']].apply(lambda x: x[1] if x[0] == '' else x[0], axis=1)
index_data = all_comments_merged[all_comments_merged['parent_main_body'].apply(lambda x: len(x.split()) > 0)][['body', 'parent_main_body', 'parent_selftext', 'Topics', 'parent_id']]
index_data = all_comments_merged[all_comments_merged['body'].apply(lambda x: r'[removed]' != x and r'[deleted]' != x)][['body', 'parent_main_body', 'parent_selftext', 'Topics', 'parent_id']]
index_data.rename(columns={'parent_main_body': 'parent_body'}, inplace=True)
index_data


In [ ]:
import json
xyz = json.loads(index_data.to_json(orient = "records"))
#del xyz['data']['index']

json_object= json.dumps(xyz)
#parsed = json.loads(xyz)
#json_object = json.dumps(parsed, indent=4)

with open("reddit_data.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
final_data = index_data.drop(['parent_selftext','parent_id'], axis=1)
final_data

In [31]:
import json
f = open('reddit_data.json')
data = json.load(f)
f.close()

In [32]:
import pandas as pd
final_df = pd.DataFrame(data)
final_df

,body,parent_body,parent_selftext,Topics,parent_id
0,"Scary has nothing to do with it, but a fundame...",Yes . I’ll be blunt. YES. The world is a sc...,,Politics,t1_i9gz6n8
1,Yes . I’ll be blunt. YES. The world is a sc...,"So yes, got it",,Politics,t1_i9gyxuu
2,"So yes, got it",Just type porn hub dot com… what do you see (h...,,Politics,t1_i9fx1c4
3,Just type porn hub dot com… what do you see (h...,"So you're ok with death threats, lies, porn, e...",,Politics,t1_i67nt50
4,You liberals sure do fuckin hate first amendment,Biden officials worry Musk will allow Trump re...,,Politics,t3_ubtplz
...,...,...,...,...,...
144287,"Voting blue to avoid red isn't a choice, it's ...",,,None,None
144288,Unpopular opinion but just calling a restauran...,,,None,None
144289,"Did I say ""all dogs to go extinct?"" \n\nThe an...",,,None,None
144290,Thank you. It’s hard to get the message out th...,,,None,None


In [33]:
del final_df["parent_id"]
del final_df["Topics"]
del final_df["parent_selftext"]

In [41]:
final_df = final_df[final_df.parent_body != '']
final_df

,body,parent_body
0,"Scary has nothing to do with it, but a fundame...",Yes . I’ll be blunt. YES. The world is a sc...
1,Yes . I’ll be blunt. YES. The world is a sc...,"So yes, got it"
2,"So yes, got it",Just type porn hub dot com… what do you see (h...
3,Just type porn hub dot com… what do you see (h...,"So you're ok with death threats, lies, porn, e..."
4,You liberals sure do fuckin hate first amendment,Biden officials worry Musk will allow Trump re...
...,...,...
129031,You’re right about people always looking for n...,My parents want me to move out or pursue furth...
129032,This is a no brainer -_-,My parents want me to move out or pursue furth...
129033,"You're 25, you make good money, you're happy w...",My parents want me to move out or pursue furth...
129034,You’re 25; not 15. Move out. Let your parents ...,My parents want me to move out or pursue furth...


In [43]:
# libraries
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer

ModuleNotFoundError: No module named 'tensorflow'